## Variación espacio-temporal precipitación total

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

https://www.section.io/engineering-education/build-ann-with-keras/

Directorio

In [1]:
import os

print('> Directorio actual: ', os.getcwd())  
os.chdir('../')
print('> Directorio actual: ', os.getcwd()) 

> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics/cerro_saroche
> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics


In [2]:
from utils.MONGO import CONEXION
from utils.UTILS import *
from datetime import datetime
import pandas as pd

from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [3]:
import warnings
warnings.filterwarnings('ignore')

Datos de precipitación 

In [4]:
park = 'cerro_saroche'

In [5]:
DIR = f'./{park}/experiments/ann/'
experiments = [DIR + x for x in os.listdir(DIR)]

In [6]:
# Resultados
pd_summary = pd.concat([pd.read_csv(x) for x in experiments if x.find('summary')!=-1 ])
pd_summary.head()

,epocas,activation,n_neurons,capas,training_mse,training_rmse,training_mae,trainig_mape,trainig_r,test_mse,test_rmse,test_mae,test_mape,test_r
0,62,"['tanh', 'tanh']","[3, 1]",2,1.125010,1.265648,0.740108,83.012647,0.077616,1.212866,1.471044,0.801445,75.364642,0.018412
0,146,"['tanh', 'softplus', 'linear']","[5, 3, 1]",3,1.114312,1.241691,0.717863,81.055836,0.095076,1.199634,1.439121,0.777822,73.418501,0.039713
0,151,"['tanh', 'softsign', 'softsign', 'linear']","[5, 10, 1]",3,1.036572,1.074482,0.695201,89.264663,0.216935,1.119315,1.252866,0.750671,80.417841,0.163996
0,169,"['tanh', 'sigmoid', 'sigmoid', 'sigmoid', 'sof...","[5, 10, 10, 5, 1]",5,1.134807,1.287786,0.714990,73.698244,0.061483,1.224995,1.500613,0.779268,67.965560,-0.001319
0,205,"['tanh', 'sigmoid', 'sigmoid', 'sigmoid', 'lin...","[5, 15, 10, 5, 1]",5,1.082115,1.170973,0.684376,78.739066,0.146614,1.166193,1.360006,0.745392,71.375343,0.092504


In [7]:
# best model
pd_best_model = pd_summary[pd_summary.trainig_r == pd_summary.trainig_r.max()]
pd_best_model

,epocas,activation,n_neurons,capas,training_mse,training_rmse,training_mae,trainig_mape,trainig_r,test_mse,test_rmse,test_mae,test_mape,test_r
0,324,"['tanh', 'sigmoid', 'sigmoid', 'sigmoid', 'lin...","[5, 10, 10, 5, 1]",5,0.965594,0.932373,0.630774,67.181892,0.320502,1.050807,1.104195,0.689043,62.585213,0.2632


Cargando datos de precipitación ANN

In [8]:
pd_precipitacion = pd.read_pickle(f'{DIR}predicciones.pkl')
pd_precipitacion.head()

,park,periodo,year,month,id_point,latitud,longitud,type,precipitacion_mm,prediction_precipitacion_mm,elevacion_media,precipitacion_narx,prediction_ann
0,cerro_saroche,1995-01-01,1995,1,1,10.31,-69.83,training,0.072351,0.108708,508.541046,0.108708,0.207416
1,cerro_saroche,1995-02-01,1995,2,1,10.31,-69.83,training,0.340843,0.299087,508.541046,0.299087,0.262076
2,cerro_saroche,1995-03-01,1995,3,1,10.31,-69.83,training,2.290730,2.657068,508.541046,2.657068,0.698307
3,cerro_saroche,1995-04-01,1995,4,1,10.31,-69.83,training,1.064486,1.260912,508.541046,1.260912,1.442033
4,cerro_saroche,1995-05-01,1995,5,1,10.31,-69.83,training,1.114330,1.177589,508.541046,1.177589,1.287615


Validación por puntos

In [15]:
list_test = []

for id in pd_precipitacion.sort_values('id_point').id_point.unique():

    data = pd_precipitacion.sort_values('periodo').query(f"id_point=={id}")

    test = metrics(observado=data['precipitacion_narx'],
                prediccion=data['prediction_ann'],)
    test['id_point'] = id

    list_test.append( pd.DataFrame(test,index=[0]) )

pd_test_id = pd.concat(list_test)
pd_test_id.to_csv(f'./{park}/data/ann_precipitacion_metricas.csv',index=False)
pd_test_id

,mape,mae,mse,rmse,r2,id_point
0,70.109823,0.673495,1.024354,1.049300,0.322102,1
0,63.428755,0.755095,1.223716,1.497482,0.249948,2
0,65.140175,0.750064,1.143630,1.307889,0.238813,3
0,60.913166,0.627247,0.988922,0.977966,0.309138,4
0,44.945096,0.574015,0.834158,0.695820,0.287891,5
0,95.149819,0.632470,0.908374,0.825143,0.313152,6
0,82.049359,0.627930,0.920137,0.846652,0.294054,7
0,70.970498,0.676454,1.034940,1.071102,0.281857,8
0,59.533485,0.549854,0.826508,0.683116,0.360069,9
0,47.689931,0.556189,0.837514,0.701430,0.373531,10


Gráfico

In [10]:
rows=5 
cols=3

column_titles = list(map(lambda x: 'Punto id:' + str(int(x)), pd_precipitacion.sort_values('id_point',ascending=True).id_point.unique().tolist() ))

fig = make_subplots(rows=rows,
                    cols=cols,
                    subplot_titles=column_titles,
                    horizontal_spacing=0.1,
                    vertical_spacing=0.13)

row_order = pd_precipitacion.sort_values('id_point',ascending=True).id_point.unique().reshape(rows,cols).tolist()

marker_line_width=1.5
marker_size=1.5
line_width=2

for row in range(len(row_order)):

    list_row = row_order[row]
    for col in range(len(list_row)):
        
        if row_order[row][col]==1:
            showlegend=True
        else:
            showlegend=False
            
            
        data_fig = pd_precipitacion.sort_values('id_point',ascending=True).query(f"id_point=={row_order[row][col]}")
        data_fig = data_fig.sort_values('periodo',ascending=True)

        fig.add_trace(
            go.Scatter(x=data_fig['periodo'],
                       #y=data_fig.total_precipitation_mm,
                       y=data_fig['precipitacion_narx'],
                       mode='lines+markers',
                       name='p(t)',# Id: {id}'.format(id=id_),
                       marker_symbol='square',
                       marker_line_width=marker_line_width,
                       marker_size=marker_size,
                       marker_line_color='#4F64B4',
                       marker_color='#4F64B4', 
                       line=dict(color='#004CFF', width=line_width),
                       legendgroup='group1',
                       showlegend=showlegend,
                       
                      ),
            row=row+1, 
            col=col+1,
                
        )
        
        fig.add_trace(
            go.Scatter(x=data_fig['periodo'],
                       #y=data_fig.total_precipitation_mm,
                       y=data_fig['prediction_ann'],
                       mode='lines+markers',
                       name='Predicción',# Id: {id}'.format(id=id_),
                       #line=dict(color='#00B4B1', width=2),
                       marker_symbol='star',
                       marker_line_width=marker_line_width,
                       marker_size=marker_size,
                       marker_line_color='#C60000',
                       marker_color='#C60000',
                       line=dict(color='#00C6BB', width=line_width,dash='dot'),
                       legendgroup='group1',
                       showlegend=showlegend,
                       
                      ),
            row=row+1, 
            col=col+1,
                
        )

        
        fig.update_xaxes(title_text='Mes',title_font=dict(size=12),
                         range=[data_fig.periodo.max() - pd.DateOffset(months=4*12),
                                data_fig.periodo.max()])
        fig.update_yaxes(title_text='Precipitación (mm)',
                         title_font=dict(size=12),
                         range=[0,6]
                        )

# fig['layout']['title']['y']=0.98
# fig['layout']['margin']['t']=100
        
#        
fig.update_xaxes(tickformat="%Y/%m",showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)        
fig.update_layout( height=800,
                   width=1500,
                   showlegend = True,
                   font = dict(size = 14),
                   template='plotly_white',
                   title_text=f"""
                             Modelo Espacio Temporal: Precipitación total periodo {pd_precipitacion.periodo.min().strftime('%Y/%m')} al {pd_precipitacion.periodo.max().strftime('%Y/%m')}
                             <br><sup>Parque {'Cerro Saroche'}</sup>
                             """,
                   #xaxis_range=['1980-01-01','2023-01-01'],
                   legend_title_text='Serie',
                   legend_title = dict( font = dict(size = 14)),
                   uniformtext_minsize=8,
                   uniformtext_mode='hide',
                   legend = dict(
                                #orientation="h",
                                yanchor="bottom",
                                y=1.05,
                                xanchor="right",
                                x=1,
                                font_size=14
                            )
                 )


fig.write_image(f'./{park}/figures/ann_precipitacion.png')
fig.show()

# Preparando datos NDVI

### Conexión Mongo DB

In [11]:
# Creando la conexión con MongoDB
db = CONEXION.conexion()
db.list_collection_names()

['meteorological', 'estimateSSTNino34', 'SSTNino34']

Descargando información

In [12]:
# Realizando consulta
meteorological = db.meteorological.find({"park":park})

# Generando pandas dataframe
pd_ndvi = pd.DataFrame([file for file in meteorological])
pd_ndvi['periodo'] = pd_ndvi.time.apply(lambda x: datetime.fromordinal(x))
pd_ndvi['mes_year'] =  pd_ndvi['periodo'].dt.strftime('%B-%Y')
#data_pandas.index = pd.to_datetime(data_pandas.periodo)

pd_ndvi.head()

,_id,id_point,park,time,elevacion_maxima,elevacion_media,elevacion_mediana,latitud,longitud,ndvi_maxima,ndvi_media,ndvi_mediana,precipitacion_mm,time_actualizacion,periodo,mes_year
0,6354801a343c9f2921d883bf,1,cerro_saroche,719163,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,0.913065,738450,1970-01-01,January-1970
1,6354801a343c9f2921d883d3,1,cerro_saroche,719194,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,0.081278,738450,1970-02-01,February-1970
2,6354801a343c9f2921d883e0,1,cerro_saroche,719222,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,0.413783,738450,1970-03-01,March-1970
3,6354801a343c9f2921d883ee,1,cerro_saroche,719253,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,0.895653,738450,1970-04-01,April-1970
4,6354801b343c9f2921d88407,1,cerro_saroche,719283,921.0,508.541046,491.0,10.31,-69.83,NaN,NaN,NaN,2.909450,738450,1970-05-01,May-1970


Gráficos

In [13]:
rows=5
cols=3

column_titles = list(map(lambda x: 'Punto id:' + str(int(x)), pd_ndvi.sort_values('id_point',ascending=True).id_point.unique().tolist() ))

fig = make_subplots(rows=rows,
                    cols=cols,
                    subplot_titles=column_titles,
                    horizontal_spacing=0.1,
                    vertical_spacing=0.13)


row_order = pd_ndvi.sort_values('id_point',ascending=True).id_point.unique().reshape(rows,cols).tolist()
    
for row in range(len(row_order)):

    list_row = row_order[row]
    for col in range(len(list_row)):
        
        if row_order[row][col]==1:
            showlegend=True
        else:
            showlegend=False

        data_fig = pd_ndvi.query(f"id_point=={row_order[row][col]}")
        data_fig = data_fig.sort_values('periodo',ascending=True)

        fig.add_trace(
            go.Scatter(x=data_fig['periodo'],
                        y=data_fig['ndvi_media'],
                        mode='lines',
                        name='NDVI',# Id: {id}'.format(id=id_),
                        line=dict(color='#008302', width=2),
                        legendgroup='group1',
                        showlegend=showlegend,
                        
                        ),
            row=row+1, 
            col=col+1
                
        )
        
        fig.update_xaxes(title_text='Mes',
                            title_font=dict(size=12),
                        range=[ data_fig.dropna().periodo.min(),
                                data_fig.periodo.max()])
        fig.update_yaxes(title_text='NDVI',
                            title_font=dict(size=12),
                            range=[0,1]
                        )
        
fig['layout']['title']['y']=0.98
fig['layout']['margin']['t']=100
        
#        
fig.update_xaxes(tickformat="%Y/%m",showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)        
fig.update_layout( height=800,
                   width=1500,
                   showlegend = True,
                   font = dict(size = 14),
                   template='plotly_white',
                   title_text=f"""
                             NDVI periodo {pd_ndvi.periodo.min().strftime("%Y/%m")} al {pd_ndvi.periodo.max().strftime("%Y/%m")}
                             <br><sup>Parque {'Cerro Saroche'}</sup>
                             """,
                #    xaxis_range=[data_fig.periodo.max() - pd.DateOffset(months=4*12),
                #                 data_fig.periodo.max()],
                  legend_title_text='Serie',
                  legend_title = dict( font = dict(size = 14)),
                   uniformtext_minsize=8,
                   uniformtext_mode='hide',
                   legend = dict(
                                #orientation="h",
                                yanchor="bottom",
                                y=1.05,
                                xanchor="right",
                                x=1,
                                font_size=14
                            )
                 )

fig.write_image(f'./{park}/figures/ndvi_{park}.png')

fig.show()

In [14]:
pd_model = pd.merge(pd_precipitacion,pd_ndvi[[ 'id_point', 'latitud','longitud', 'ndvi_media','periodo']],
                    on=['periodo','id_point','latitud','longitud'],
                    how='left')

pd_model.to_pickle(f'./{park}/data/ann_precipitacion.pkl')